In [ ]:
%pip install --upgrade tdmclient

In [5]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

Run les 2 cellules suivantes pour vérifier que le robot roule droit et checker qu'il est pas bourré comme le mien...

In [6]:
motor_left_target = 55
motor_right_target = 55

In [7]:
motor_left_target = 0
motor_right_target = 0

Run ce programme pour faire la local obstacle avoidance

In [ ]:
%%run_python --wait

SPEED0 = 50       # nominal speed
OBST_THR_L = 10      # low obstacle threshold to switch state 1->0
OBST_THR_H= 20      # high obstacle threshold to switch state 0->1
OBST_SPEED_GAIN = 10  # /100 (actual gain: 5/100=0.05)
SENSOR_SCALE = 200  # Scale factors for sensors

state = 0          # 0= go to goal, 1=obstacle avoidance
obst = [0,0,0,0,0]   # measurements from the 2 left and the 2 right prox sensors

timer_period[0] = 10   # 10ms sampling time

proxsens1 = 0   #proximity sensor 1
proxsens2 = 1   #proximity sensor 2
proxsens3 = 2   #proximity sensor 3
proxsens4 = 3   #proximity sensor 4
proxsens5 = 4   #proximity sensor 5

statego2goal = 0
stateobstavoid = 1
 

@onevent 
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target, state, obst, obstThrH, obstThrL, obstSpeedGain, speed0, speedGain, motor_right_speed, motor_left_speed
    # acquisition from the proximity sensors to detect obstacles
    obst = [prox_horizontal[proxsens1], prox_horizontal[proxsens2], prox_horizontal[proxsens3], prox_horizontal[proxsens4], prox_horizontal[proxsens5]]
    mean_obst_left = (obst[0] + obst[1])//2
    mean_obst_right = (obst[3] + obst[4])//2
    obst_front = obst[2]

    # tdmclient does not support yet multiple and/or in if statements:
    if state == statego2goal: 
        # switch from goal tracking to obst avoidance if obstacle detected
        if mean_obst_right > OBST_THR_H:
            state = 1
        elif mean_obst_left > OBST_THR_H:
            state = 1
    elif state == stateobstavoid:
        if  mean_obst_right < OBST_THR_L:
            if mean_obst_left < OBST_THR_L: 
                # switch from obst avoidance to goal tracking if obstacle got unseen
                state = 0
                
    if  state == stateobstavoid:
        # obstacle avoidance: accelerate wheel near obstacle
        obst_in_front = 1
        
        if (abs(motor_left_speed - motor_right_speed) < 50):
            if obst_front > 500:
                motor_left_target = -150
                motor_right_target = 150
            else:
                obst_in_front = 0
                 
        if obst_in_front==0:
            motor_left_target = SPEED0 + OBST_SPEED_GAIN * ( mean_obst_left // 100)
            motor_right_target = SPEED0 + OBST_SPEED_GAIN * ( mean_obst_right // 100)
            
    elif state == statego2goal:
        
        motor_left_target = 200
        motor_right_target = 200

        

Exception in thread Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
Thread-9:
Traceback (most recent call last):
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39\lib\site-packages\tdmclient\tcp.py", line 65, in run
    self.run()
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39\lib\site-packages\tdmclient\tcp.py", line 65, in run
    packet = self.read_packet()
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39\lib\site-packages\tdmclient\tcp.py", line 58, in read_packet
    packet = self.read_packet()
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39\lib\site-packages\tdmclient\tcp.py", line 58, in read_packet
    raise error
  File "C:\Users\ision\AppData\Local\Programs\Python\Python39

In [ ]:
await tdmclient.notebook.stop()   